In [1]:
import networkx as nx
import os
import pandas as pd
from utils.global_rules import rules

dataDir = "../data/"
cacheDir = "../data/cache/"

In [4]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

test_evidence = rules["evidence"]["1"]
G = process.get_graph()

Loading graph...
Initial Node shape:  (2371558, 4)
Initial Link shape:  (3286986, 3)
After drop nan and duplicates:
Node shape:  (2371558, 4)
Link shape:  (3285490, 3)
Graph loaded.


In [3]:
from networkx.algorithms.community import louvain_communities

G = nx.karate_club_graph()

communities = louvain_communities(G)

for community in communities:
    print(community)

{16, 4, 5, 6, 10}
{0, 1, 2, 3, 7, 11, 12, 13, 17, 19, 21}
{32, 33, 8, 9, 14, 15, 18, 20, 22, 26, 29, 30}
{23, 24, 25, 27, 28, 31}


In [16]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

test_reachable_nodes = process.get_subgrah(G, test_evidence)
test_subgraph = G.subgraph(test_reachable_nodes).copy()
G_filtered = process.filter_nodes(test_subgraph)

# get nodes from G_filtered
nodes = G_filtered.nodes(data=True)

# filter nodes with is_core = True
core_nodes = list(filter(lambda x: x[1]["is_core"], nodes))
# get node id
core_nodes = set(map(lambda x: x[0], core_nodes))
print("Number of core nodes: ", len(core_nodes))

# get subgraph with core nodes
core_subgraph = G_filtered.subgraph(core_nodes).copy()

# drop isolated nodes
core_subgraph.remove_nodes_from(list(nx.isolates(core_subgraph)))

# plot core subgraph
process.plot_graph(core_subgraph)

Filtering nodes...
Removing nodes by count...
Nodes removed:  0 Nodes left:  30767
Removing nodes by pagerank...
Nodes removed:  29767 Nodes left:  1000
Number of core nodes:  331


In [13]:
# export core subgraph to json
import json

data = nx.readwrite.json_graph.node_link_data(core_subgraph)
with open(os.path.join(cacheDir, "core_subgraph.json"), "w") as f:
    json.dump(data, f, indent=4)